Имеются данные с информацией о событиях, которые произошли с объявлением 121288 за два дня. Необходимо посчитать метрики (количество показов, количество кликов, CTR, сумма потраченных денег ) в разрезе каждого дня и их изменение в процентах, и отправить отчет в бот Telegram, отчет должен отправляться автоматически каждый понедельник в 12-00.

Вид отчета:          
Отчет по объявлению 121288 за 2 апреля           
Траты: Х рублей (+Y%)            
Показы: X (+Y%)            
Клики: X (-Y%)            
CTR: X (-Y%)           

In [3]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime
import pandas as pd
import requests
import json
from urllib.parse import urlencode

def send_report():
    data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ti6Su94955DZ4Tky8EbwifpgZf_dTjpBdiVH0Ukhsq94jZdqoHuUytZsFZKfwpXEUCKRFteJRc9P/pub?gid=889004448&single=true&output=csv')

    ad_cost = data.ad_cost[0]
    ad_id = data.ad_id[0]
    
#     Transform data to compute metrics:

    data1 = data.pivot_table(values='ad_id',index='date',columns='event',aggfunc='count')
    data1['CTR'] = data1.click / data1.view * 100
    data1['sum'] = ad_cost / 1000 * data1.view
    data1 = data1.transpose()

    data1['percent'] = data1[data1.columns[1]] / data1[data1.columns[0]] * 100 - 100

    data1 = data1.round().astype('int')
    
#     Setting up report message: 

    message = (f"Отчет по объявлению {ad_id} за {data1.columns[1]}:\n\
    Траты: {data1.loc['sum', data1.columns[1]]} ({data1.loc['sum','percent']} %)\n\
    Показы: {data1.loc['view', data1.columns[1]]} ({data1.loc['view','percent']} %)\n\
    Клики: {data1.loc['click', data1.columns[1]]} ({data1.loc['click','percent']} %)\n\
    CTR: {data1.loc['CTR', data1.columns[1]]} ({data1.loc['CTR','percent']} %)")
    
#     Writing message into the new text file:

    filepath = f'report for {data1.columns[1]}.txt'

    with open(filepath,'w') as f:
        f.write(message)
        
#      Sending params for Telegram:

    token = '***'
    chat_id = ***  

    params = {'chat_id': chat_id, 'text': message}
    base_url = f'https://api.telegram.org/bot{token}/'

#       Sending text message to bot chat in Telegram:

    url = base_url + 'sendMessage?' + urlencode(params)
    resp = requests.get(url)

#     OR to send text file (.txt) to bot chat in Telegram:

    url = base_url + 'sendDocument?' + urlencode(params)
    files = {'document': open(filepath, 'rb')}
    resp = requests.get(url, files=files)

# Airflow DAG settings:

default_args = {'owner':'d.kandy',
                'start_date':datetime(2020,12,1)}

dag = DAG(dag_id = 'Miniproject_16_Airflow',
          default_args = default_args,
          schedule_interval='0 12 * * 1')

task = PythonOperator(task_id='making_report',
                     python_callable=send_report,
                     dag=dag)